In [1]:
import imapclient
import email

In [14]:
server = imapclient.IMAPClient(host='mail.archive.org', port='993')
server.login('support.tickets.bot@archive.org', 'YmGG6g45akGdV5q0QX8it3CDXSl7Vc')

b'Logged in'

In [17]:
server.select_folder('FROM_ARCHIVE_ACCOUNTS')
ids = server.search(['BEFORE', '28-Nov-2020'])
ids = server.search(['ALL'])
len(ids)

1052

In [18]:
# server.delete_messages(ids)
# server.expunge()

(b'Expunge completed.',
 [(1052, b'EXPUNGE'),
  (1051, b'EXPUNGE'),
  (1050, b'EXPUNGE'),
  (1049, b'EXPUNGE'),
  (1048, b'EXPUNGE'),
  (1047, b'EXPUNGE'),
  (1046, b'EXPUNGE'),
  (1045, b'EXPUNGE'),
  (1044, b'EXPUNGE'),
  (1043, b'EXPUNGE'),
  (1042, b'EXPUNGE'),
  (1041, b'EXPUNGE'),
  (1040, b'EXPUNGE'),
  (1039, b'EXPUNGE'),
  (1038, b'EXPUNGE'),
  (1037, b'EXPUNGE'),
  (1036, b'EXPUNGE'),
  (1035, b'EXPUNGE'),
  (1034, b'EXPUNGE'),
  (1033, b'EXPUNGE'),
  (1032, b'EXPUNGE'),
  (1031, b'EXPUNGE'),
  (1030, b'EXPUNGE'),
  (1029, b'EXPUNGE'),
  (1028, b'EXPUNGE'),
  (1027, b'EXPUNGE'),
  (1026, b'EXPUNGE'),
  (1025, b'EXPUNGE'),
  (1024, b'EXPUNGE'),
  (1023, b'EXPUNGE'),
  (1022, b'EXPUNGE'),
  (1021, b'EXPUNGE'),
  (1020, b'EXPUNGE'),
  (1019, b'EXPUNGE'),
  (1018, b'EXPUNGE'),
  (1017, b'EXPUNGE'),
  (1016, b'EXPUNGE'),
  (1015, b'EXPUNGE'),
  (1014, b'EXPUNGE'),
  (1013, b'EXPUNGE'),
  (1012, b'EXPUNGE'),
  (1011, b'EXPUNGE'),
  (1010, b'EXPUNGE'),
  (1009, b'EXPUNGE'),
  (1008,

In [56]:
data = server.fetch(ids, ['ENVELOPE', 'INTERNALDATE', 'BODY[TEXT]'])

In [57]:
previous_dt = None
for msg_id in data:
    current_dt = data[msg_id][b'INTERNALDATE']
    if current_dt is None:
        print(msg_id, data[msg_id])
    if previous_dt is not None:
        if current_dt < previous_dt:
            print('found one')
    
    previous_dt = current_dt

found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one


In [59]:
data[85]

{b'SEQ': 84,
 b'INTERNALDATE': datetime.datetime(2020, 11, 24, 13, 6, 46),
 b'ENVELOPE': Envelope(date=datetime.datetime(2020, 11, 24, 13, 6, 42), subject=b'Automatische Antwort: New Life for Flash Animations on the Internet Archive', from_=(Address(name=b'Rothermundt Christian HCARE-KSSG-ONKO', route=None, mailbox=b'Christian.Rothermundt', host=b'kssg.ch'),), sender=(Address(name=b'Rothermundt Christian HCARE-KSSG-ONKO', route=None, mailbox=b'Christian.Rothermundt', host=b'kssg.ch'),), reply_to=(Address(name=b'Rothermundt Christian HCARE-KSSG-ONKO', route=None, mailbox=b'Christian.Rothermundt', host=b'kssg.ch'),), to=(Address(name=b'Internet Archive', route=None, mailbox=b'info', host=b'archive.org'),), cc=None, bcc=None, in_reply_to=b'<38bd6154386f64fcd92204a25.0b0242ee69.20201124174939.b43af12478.5e9e9ff7@mail20.sea41.rsgsv.net>', message_id=b'<cb0d2b719b16404fb7a54899cbfba84a@s-000-0140.sg.hcare.ch>'),
 b'BODY[TEXT]': b'--_000_cb0d2b719b16404fb7a54899cbfba84as0000140sghcarech_\r\nC

# Difflib

In [1]:
import difflib

In [3]:
isjunk = lambda c: c in ' \t\n'

In [5]:
a = """
if you’re comparing lines as sequences of characters, and don’t want to synch up on blanks or hard tabs.

The optional arguments a and b are sequences to be compared; both default to empty strings. The elements of both sequences must be hashable.

The optional argument autojunk can be used to disable the automatic junk heuristic.

New in version 3.2: The autojunk parameter.

SequenceMatcher objects get three data attributes: bjunk is the set of elements of b for which isjunk is True; bpopular is the set of non-junk elements considered popular by the heuristic (if it is not disabled); b2j is a dict mapping the remaining elements of b to a list of positions where they occur. All three are reset whenever b is reset with set_seqs() or set_seq2().

New in version 3.2: The bjunk and bpopular attributes."""

b = """
if you’re comparing lines as sequences of characters, and don’t want to synch up on blanks or hard tabs.
\n
\nThe optional arguments a and b a sequences to be copared; both default to empty strings. The elements of both sequences must be hashable.
\n
The optional argument autojunk can be used to disable the automatic junk heuristic.
asd
\nNew in version 3.2: The autojunk parameter.

SequenceMatcher objects get three data attributes: bjunk is the set of elements of b for which isjunk is True; bpopular is the set of non-junk elements considered popular by the heuristic (if it is not disabled); b2j is a dict mapping the remaining elements of b to a list of positions where they occur. All three are reset whenever b is reset with set_seqs() or set_seq2().

New in version 3.2: The bjunk and bpopular attributes."""

In [6]:
m = difflib.SequenceMatcher(isjunk=isjunk, a=a, b=b, autojunk=False)

In [7]:
m

In [9]:
m.get_matching_blocks()

[Match(a=0, b=0, size=107),
 Match(a=107, b=109, size=32),
 Match(a=141, b=141, size=19),
 Match(a=161, b=160, size=88),
 Match(a=249, b=249, size=84),
 Match(a=333, b=337, size=476),
 Match(a=809, b=813, size=0)]

In [10]:
m.ratio()

0.9938347718865598